In [1]:
from data_180218 import load_data
from spot2intensity import create_circle_patches
import matplotlib.pyplot as plt
from matplotlib import patches
from utils import ensure_dir
import numpy as np
import pickle

In [2]:
collections = load_data()


In [3]:
#c_names = ('l', 'm', 'n')
#sub_collection = {x: collections[x] for x in c_names if x in collections}

In [4]:
for c_name, c_data in collections.iteritems():
    if  c_name in ["slide_27"] :
        directory = "data/{}/{}/".format(c_data.study,c_name)
        ensure_dir(directory)
        print("calculating:{}".format(c_name))
        spots = c_data.pd_complete_spots()
        with open(directory+"spots_class", 'w') as f:
            pickle.dump(spots,f)
            
        spots.plot_grid(on="b_im")
        plt.savefig(directory+"im_grid_before_600_80_635.jpg",bbox_inches='tight')
        spots.plot_grid(on="a_im")
        plt.savefig(directory+"im_grid_after_600_80_635.jpg",bbox_inches='tight')
        plt.close()
        
        spots.df[["Block","Row","Column","Name","std_intensities2","intensities2","intensities","intensities2_b","circle_qual"]].to_csv("{}spots.tsv".format(directory), sep='\t',)
        inte = spots.df.pivot(index="Row", columns="Column", values="intensities")
        inte.to_csv("{}intensity.tsv".format(directory), sep='\t')
        int2 = spots.df.pivot(index="Row", columns="Column", values="intensities2")
        int2.to_csv("{}intensity2.tsv".format(directory), sep='\t')
        circ_q = spots.df.pivot(index="Row", columns="Column", values="circle_qual")
        circ_q.to_csv("{}circle_quality.tsv".format(directory), sep='\t')
        std2 = spots.df.pivot(index="Row", columns="Column", values="std_intensities2")
        std2.to_csv("{}intensity2_std.tsv".format(directory), sep='\t')



calculating:slide_27
Claculating grid starting at:(42,144)


In [5]:
c_names = ["slide_19","slide_20","slide_46","slide_47"]
sub_collections = collections
for c_name in c_names:
    sub_collections.pop(c_name)

for c_name, c_data in sub_collections.iteritems():
    directory = "data/{}/{}/".format(c_data.study,c_name)
    with open(directory+"spots_class", 'r') as f:
        spots = pickle.load(f)
    spots.df.boxplot(vert=False,column="intensities2", by="Name", figsize=(20,20))
    plt.xlabel("Intensities")  
    plt.ylabel("Peptides")
    plt.title(c_name+" mean on circle")
    plt.savefig("data/{}/correlation_plots/array/circ/{}_boxplot.png".format(c_data.study,c_name))
    plt.close()
    

    

/home/janekg89/Envs/flutype_webapp/local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [6]:
np.sort(sub_collections.keys())


array(['slide_21', 'slide_22', 'slide_23', 'slide_24', 'slide_25',
       'slide_26', 'slide_27', 'slide_28', 'slide_29', 'slide_30',
       'slide_31', 'slide_32', 'slide_33', 'slide_34', 'slide_35',
       'slide_36', 'slide_37', 'slide_38', 'slide_39', 'slide_40',
       'slide_41', 'slide_42', 'slide_44', 'slide_45'],
      dtype='|S8')

In [7]:
#a1 = spot_dict[c_names[0]]
#a2 = spot_dict[c_names[1]]


#plt.scatter(a1.df["intensities"],a2.df["intensities"])
#plt.xlabel(c_names[0])
#plt.xlabel(c_names[1])
#plt.xlim(left=20000)
#plt.ylim(bottom=20000)

#plt.xscale("log")
##plt.yscale("log")

#plt.show()

In [8]:
#a1.df.loc[all(a1.df["Row"]==4 & a1.df["Column"]==1)].spot_images

#a_1_n = a1.df.loc[a1.df["Column"]==1]
#a_1_n =a_1_n.loc[a_1_n["Row"]==4]
#print(a_1_n.spot_images.values[0])
#plt.imshow(a_1_n.spot_images.values[0])
#plt.show()

In [9]:
import pandas as pd

In [10]:
frames = []
for c_name, c_data in sub_collections.iteritems():
    directory = "data/{}/{}/".format(c_data.study,c_name)
    df = pd.read_csv("{}spots.tsv".format(directory), sep='\t',)
    df["Collection"] = c_name
    frames.append(df)
    

merged = pd.concat(frames)




In [11]:
merged.head()

,ID,Block,Row,Column,Name,std_intensities2,intensities2,intensities,intensities2_b,circle_qual,Collection
0,1,1,1,1,LF,4008.278466,3872.700125,9010898,428.925972,0.593750,slide_28
1,2,1,1,2,LF,2353.198870,1848.082811,4527545,388.346299,0.802083,slide_28
2,3,1,1,3,LF,2170.016927,1371.328733,3664968,336.179423,0.614583,slide_28
3,4,1,1,4,LF,1865.211575,1031.460477,3219597,348.723965,0.645833,slide_28
4,5,1,1,5,LF,1885.540587,1038.954831,2774662,483.819322,0.510417,slide_28


In [12]:
directory = "data/{}/pep_plots/".format(c_data.study)

# TODO: different colors for the Viruses /background
# TODO: add method to title PEP_76 (intensities, sum square method, no background)
# TODO: Add virus to ylabel (V1 (45, 1))
#(V1 (45, 0.5))
# boxprops = {'color': 'red'}


ensure_dir(directory)
for pep_name, pep_data in merged.groupby("Name"):
    print(pep_name)
    pep_data.sort_values(by="Collection",ascending=True, inplace=True)
    pep_data.boxplot(vert=False,column="intensities2", by="Collection", figsize=(10,10), boxprops=boxprops)
    plt.xlabel("Intensities")
    plt.ylabel("Collections")
    plt.title("{},mean_on_circle".format(pep_name))
    plt.savefig(directory+"mean/"+pep_name+".png")
    plt.close()

AK Cal 28666


/home/janekg89/Envs/flutype_webapp/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


NameError: name 'boxprops' is not defined

In [ ]:
# plt.plot(spots.df.Name, spots.df.intensities2, 'o')
#plt.figure(figsize=(20,20))

# TODO: save all in one folder (array_plots)
# TODO: add methods to title

#plt.boxplot(a1.df.Name, a1.df.intensities, 'o')
a1.df.boxplot(vert=False,column="intensities2", by="Name", figsize=(20,20))
#fig ,axes = plt.subplots(1,figsize=(20,20))
#for name, data in a1.df.groupby("Name"):
plt.xlabel("Intensities")  
plt.ylabel("Peptides")
plt.title(c_names[0])
#plt.yscale("log")
plt.show()

In [ ]:

directory = "data/{}/correlation_plots/".format(c_data.study)
ensure_dir(directory)
for pep_name, pep_data in merged.groupby("Name"):
    print(pep_name)
    pep_data.sort_values(by="Collection",ascending=True, inplace=True)
    pep_data.boxplot(vert=False,column="intensities", by="Collection", )#figsize=(20,30))
    plt.xlabel("Intensities")
    plt.ylabel("Collections")
    plt.title(pep_name)
    plt.savefig(directory+pep_name+".png")
    plt.close()

In [ ]:
viruses = [1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5]
merged["virus"] = 0
vir1 = [c in ["slide_21","slide_22","slide_23","slide_24","slide_25"] for c in merged["Collection"]]
vir2 = [c in ["slide_26","slide_27","slide_28","slide_29","slide_30"] for c in merged["Collection"]]
vir3 = [c in ["slide_31","slide_32","slide_33","slide_34","slide_35"] for c in merged["Collection"]]
vir4 = [c in ["slide_36","slide_37","slide_38","slide_39","slide_40"] for c in merged["Collection"]]
vir5 = [c in ["slide_41","slide_42","slide_43","slide_44","slide_45"] for c in merged["Collection"]]

merged["concentration"] = 1
con2=[c in ["slide_24","slide_29","slide_34","slide_39","slide_44"] for c in merged["Collection"]]
con05=[c in ["slide_25","slide_30","slide_35","slide_40","slide_45"] for c in merged["Collection"]]

merged["concentration"].loc[con2]=2
merged["concentration"].loc[con05]=0.5

merged["virus"].loc[vir1] = 1
merged["virus"].loc[vir2] = 2
merged["virus"].loc[vir3] = 3
merged["virus"].loc[vir4] = 4
merged["virus"].loc[vir5] = 5

## bad collection

cor_data = merged.loc[merged["Collection"] != "slide_38"]
    #c_data["virus"] = virus

In [ ]:
c1 = cor_data.loc[cor_data["Collection"]=="slide_22"]
c2 = cor_data.loc[cor_data["Collection"]=="slide_21"]
plt.scatter(c1["intensities"],c2["intensities"])
plt.show()


In [ ]:
#print(len(list(combinations(collections.keys(),2))))
con1_data = cor_data.loc[cor_data["concentration"] == 1]
frames = []
for name, data in con1_data.groupby(["virus","Name"]):
    x = data.median()
    x["intensities_mean_std"] = data["intensities"].std()
    x["intensities2_mean_std"] = data["intensities2"].std()

    x.name = name
    frames.append(x)
    
    
mean_data = pd.concat(frames, axis=1)

In [ ]:
mean_data = mean_data.transpose().reset_index()
mean_data = mean_data.rename(columns={"level_1":"Peptide Batch","level_0":"Virus"})


In [ ]:
from itertools import combinations
plt.close()
virus_combinations = list(combinations(range(1,6),2))
plt.figure(figsize=(15,15))
plt.cmap =plt.get_cmap("Paired")
directory = "data/{}/correlation_plots/".format(c_data.study)
ensure_dir(directory)
for v1,v2 in virus_combinations:
    print(v1,v2)
    if (v1==2 and v2==4):
        v1_data = mean_data.loc[mean_data["Virus"]==v1]
        v2_data = mean_data.loc[mean_data["Virus"]==v2]
        x = v1_data["intensities2"]
        y = v2_data["intensities2"]
        x_err = v1_data["intensities2_mean_std"]
        y_err = v2_data["intensities2_mean_std"]

        plt.errorbar(x=x,xerr=x_err,y=y,yerr=y_err, label="v1={},v2={}".format(v1,v2),fmt="o",linestyle=" ",alpha=.4, capthick=2)
    #plt.title((v1,v2))
#plt.xscale("log")
#plt.yscale("log")
plt.legend()
plt.title("mean on virus with same concentration, v1/v2 correlation, intensities mean on circle")
plt.xlabel("v1")
plt.ylabel("v2")
plt.show()
#mean_data

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(np.asarray(c_data.tifs_a[1]), cmap="gray")
plt.show()

In [ ]:
plt.plot(spots.df.Name, spots.df.intensities2)